In [1]:
from csoundengine import *
import logging
# logging.basicConfig(level="DEBUG")


In [2]:
e = Engine()
s = e.session()




--Csound version 6.19 (double samples) Jan 19 2025
[commit: 2536da284dd70ec7272040cb0763f70ae57123c4]
libsndfile-1.2.0
sr = 48000.0, kr = 750.000, ksmps = 64
0dBFS level = 1.0, A4 tuning = 442.0
audio buffered in 256 sample-frame blocks
reading 2048-byte blks of shorts from adc:Built-in Audio Analog Stereo: (RAW)
writing 512 sample blks of 64-bit floats to dac:Built-in Audio Analog Stereo:
SECTION 1:


In [3]:
foo = s.defInstr('foo', r'''
; foo instr
; Args:
;   kfreq: frequency
if changed2(kfreq) == 1 then
    println "kfreq: %f", kfreq
endif
outch 1, oscili:a(0.1, kfreq) * linsegr:a(0, 0.01, 1, 0.01, 0)
''', args={'kfreq': 800})
foo

Instr(foo, kfreq=800)

In [4]:
print(foo.generateBody())

; foo instr
; Args:
;   kfreq: frequency
kfreq = p5
if changed2(kfreq) == 1 then
    println "kfreq: %f", kfreq
endif
outch 1, oscili:a(0.1, kfreq) * linsegr:a(0, 0.01, 1, 0.01, 0)
atstop dict_get:i(gi__builtinInstrs, "notifyDealloc"), 0, 0, p1


In [7]:
sy = s.sched('foo', 0, 10, kfreq=800)
# sy.engine.callLater(0.5, sy.stop)
sy

Button(description='Stop', style=ButtonStyle())

Output()

Synth(𝍪 foo=501.0003 start=8.201 dur=10.000 p5=800)

In [6]:
sy.stop()


In [8]:
sy.set(kfreq=502)

kfreq: 502.000000


In [9]:
foo = s.defInstr('foo', r'''
{kfreq=1000}
if changed2(kfreq) == 1 then
    println "kfreq: %f", kfreq
endif
outch 1, oscili:a(0.1, kfreq) * linsegr:a(0, 0.01, 1, 0.01, 0)
''')
foo

Instr(foo, kfreq=1000)

In [10]:
foo.dynamicParamNames()

frozenset({'kfreq'})

In [11]:
sy = s.sched('foo', kfreq=700)
sy

Button(description='Stop', style=ButtonStyle())

Output()

Synth(𝍪 foo=501.0004 start=28.009 dur=-1.000 p5=700)

In [12]:
sy.args

[700.0]

In [13]:
sy.set(kfreq=3140)

kfreq: 3140.000000


In [14]:
print(sy.body)

kfreq = p5
if changed2(kfreq) == 1 then
    println "kfreq: %f", kfreq
endif
outch 1, oscili:a(0.1, kfreq) * linsegr:a(0, 0.01, 1, 0.01, 0)
atstop 1, 0.01, 0, p1


In [15]:
bar = s.defInstr('bar', r'''
|kfreq=1000|
if changed2(kfreq) == 1 then
    println "kfreq: %f", kfreq
endif
outch 1, oscili:a(0.1, kfreq) * linsegr:a(0, 0.01, 1, 0.01, 0)
''')
bar

Instr(bar, kfreq=1000)

In [17]:
baz = s.defInstr('baz', r'''
pset p1, p2, p3, 0, 440, 0.1
kfreq = p5
iamp = p6
outch 1, oscili:a(iamp, kfreq) * linsegr:a(0, 0.01, 1, 0.01, 0)
''')
baz

Instr(baz, kfreq=440, iamp=0.1)

In [23]:
def callback(*args):
    # import time
    print("**************", args)
    # time.sleep(0.01)

sy = s.sched('baz', dur=2, kfreq=800, iamp=0.1, whenfinished=lambda *args: print("*********", args))
sy

Button(description='Stop', style=ButtonStyle())

Output()

Synth(𝍪 baz=502.0007 start=138.505 dur=2.000 p5=800 kfreq=0.1)

********* (502.0007,)


In [17]:
sy.playing()

True

In [18]:
sy.set(kfreq=850)

In [16]:
sy = s.sched('baz', dur=3, kfreq=1000, iamp=0.01)
sy.automate('kfreq', (0, 0, 2, 200, 3, 800), overtake=True)

ValueError: Instrument 'baz' not defined. Known instruments: .sine, .testAudio, .playSample, .diskin, .playbuf, .playPartials, foo, bar

This should not work

In [22]:
test = s.defInstr('test', r'''
|kfreq=1000|
pset 0, 0, 0, 0, 0.1, 0.5
iamp = p5
outch 1, oscili:a(iamp, kfreq * p6)
''')
test

ValueError: Using p-fields together with named arguments is not supported. P-fields used: {5, 6}, args: {'kfreq': 1000.0}

In [23]:
test = s.defInstr('output', r'''
|ichan=1|
ain = inch:a(ichan)
aenv = follow2(ain, 0.1, 0.2)
kenv = downsamp(aenv, 16)
if changed(kenv) == 1 then
    println "kenv: %f dB", dbamp(kenv)
endif
''')
test

Instr(output, ichan:5=1)

In [4]:
s.defInstr('sine', r'''
        |iamp=0.1, kfreq=1000|
        a0 = oscili:a(kamp, kfreq)
        outch 1, a0
    ''')

Instr(sine, iamp:5=0.1, tabargs={'kfreq': 1000.0})

In [9]:
sine = s.getInstr('sine')
r = s.makeRenderer()
print(r.instrGenerateBody(sine))

; --- start generated code for dynamic args
i__token__ = p4
i__tabnum__ = gi__controlTable
i__slot__ = _getControlSlot(i__token__)
i__slicestart__ = i__slot__ * gi__controlArgsPerInstr
atstop "_releaseControlToken", 0, 0, i__itoken__

kfreq tab i__slicestart__ + 0, i__tabnum__
; --- end generated code
iamp = p5
a0 = oscili:a(kamp, kfreq)
outch 1, a0


In [11]:
r = s.makeRenderer()
r.sched('sine', dur=1, iamp=0.01, kfreq=440)
r.writeCsd("~/tmp/sine.csd")

In [14]:
with s.rendering('~/tmp/test.wav'):
    s.sched('sine', dur=1, iamp=0.01, kfreq=440)

error:  Variable 'i__slicestart__' used before defined
Line 12

error:  Variable type for i__slicestart__ could not be determined.

Parsing failed due to syntax errors
Stopping on parser failure
compileOrc error: 
001  
002  instr 502  
003      atstop 1, 0.01, 0.0, p1
004  ; --- start generated code for dynamic args
005  iamp = p5
006  
007  i__tabnum__ chnget ".dynargsTabnum"
008  if i__tabnum__ == 0 then
009      initerror sprintf("Session table does not exist (p1: %f)", p1)
010      goto __exit
011  endif
012  
013  kfreq tab i__slicestart__ + 0, i__tabnum__
014  ; --- end generated code
015  a0 = oscili:a(kamp, kfreq)
016  outch 1, a0
017  __exit:
018  endin
Could not compile code


RenderError: No score to render (start: 0.0, end: 0.0)

In [9]:
instr = s.defInstr('pfields', r'''
kfreq = p5
outch 1, oscili:a(0.1, kfreq)
''')
instr

Instr(pfields, kfreq:5)

In [10]:
instr.parsedCode

ParsedInstrBody(pfieldIndexToName={5: 'kfreq'}, pfieldLines=['kfreq = p5'], body='outch 1, oscili:a(0.1, kfreq)', lines=['', 'kfreq = p5', 'outch 1, oscili:a(0.1, kfreq)'], pfieldIndexToValue={}, pfieldsUsed={5}, outChannels={1})

In [11]:
instr.parsedCode.pfieldNameToValue

{'kfreq': 0.0}

In [12]:
instr.parsedCode.pfieldIndexToValue

{}

In [9]:
instr.dynamicPfields()

{'kfreq': 0.0}

In [10]:
instr.aliases

In [12]:
instr.args

{}